In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 300)
pd.set_option('display.width', 200)

: 

In [ ]:
# ---------------------------
# 0. Configuration
# ---------------------------
DATA_DIR = Path("data")
BENEFICIARY_FILE = DATA_DIR / "Train_Beneficiarydata.csv"
INPATIENT_FILE   = DATA_DIR / "Train_Inpatientdata.csv"
OUTPATIENT_FILE  = DATA_DIR / "Train_Outpatientdata.csv"
LABELS_FILE      = DATA_DIR / "Train_labels.csv"

In [ ]:
# ---------------------------
# 1. Helpers
# ---------------------------
def safe_read_csv(path):
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    df = pd.read_csv(path)
    print(f"Loaded '{path.name}' with shape {df.shape}")
    return df

def parse_dates(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors='coerce')

def print_missing_summary(df, name, top_n=10):
    miss = (df.isnull().mean()*100).sort_values(ascending=False)
    print(f"\nMissingness summary for {name} (top {top_n}):")
    print(miss.head(top_n).round(2))

def print_dup_counts(df, name):
    dup = df.duplicated().sum()
    print(f"{name}: {dup} duplicate rows")

def safe_info(df, name):
    print(f"\n-- {name} info --")
    print(df.info(verbose=False))
    print_missing_summary(df, name)
    print_dup_counts(df, name)

In [ ]:
# ---------------------------
# 2. Load Data
# ---------------------------
bene_df = safe_read_csv(BENEFICIARY_FILE)
inp_df  = safe_read_csv(INPATIENT_FILE)
out_df  = safe_read_csv(OUTPATIENT_FILE)
prov_df = safe_read_csv(LABELS_FILE)

# Quick inspection (first rows)
print("\nBeneficiary sample:")
display(bene_df.head(2))
print("\nInpatient sample:")
display(inp_df.head(2))
print("\nOutpatient sample:")
display(out_df.head(2))
print("\nLabels sample:")
display(prov_df.head(2))


In [ ]:
# ---------------------------
# 3. Basic Validation & Date Parsing
# ---------------------------
# We know inpatient/outpatient have: ClaimStartDt, ClaimEndDt, AdmissionDt, DischargeDt
date_cols_common = ['ClaimStartDt', 'ClaimEndDt', 'AdmissionDt', 'DischargeDt', 'DOB', 'DOD']
parse_dates(bene_df, date_cols_common)
parse_dates(inp_df, date_cols_common)
parse_dates(out_df, date_cols_common)

# Print basic info
safe_info(bene_df, "Beneficiary")
safe_info(inp_df,  "Inpatient")
safe_info(out_df,  "Outpatient")
safe_info(prov_df, "Labels")

In [ ]:
# ---------------------------
# 4. Logical Date Checks & Data Quality
# ---------------------------
print("\n--- Date consistency checks ---")

# Inpatient & Outpatient: ClaimStartDt <= ClaimEndDt
for name, df in [("Inpatient", inp_df), ("Outpatient", out_df)]:
    if 'ClaimStartDt' in df.columns and 'ClaimEndDt' in df.columns:
        bad = df[df['ClaimStartDt'] > df['ClaimEndDt']]
        print(f"{name}: {len(bad)} claims where ClaimStartDt > ClaimEndDt")

# Claims after death (merge claim-level with DOD from beneficiary)
if 'DOD' in bene_df.columns:
    temp_in = inp_df.merge(bene_df[['BeneID', 'DOD']], on='BeneID', how='left')
    cad_in = temp_in[(temp_in['DOD'].notna()) & (temp_in['ClaimStartDt'] > temp_in['DOD'])]
    print(f"Inpatient claims after death: {len(cad_in)}")
    
    temp_out = out_df.merge(bene_df[['BeneID', 'DOD']], on='BeneID', how='left')
    cad_out = temp_out[(temp_out['DOD'].notna()) & (temp_out['ClaimStartDt'] > temp_out['DOD'])]
    print(f"Outpatient claims after death: {len(cad_out)}")
else:
    print("DOD not present in beneficiary data; skipping claims-after-death check.")


In [ ]:
# ---------------------------
# 5. Beneficiary-level Feature Engineering
# ---------------------------
# Age, ChronicCount, DaysToDeath
print("\n--- Beneficiary feature engineering ---")
if 'DOB' in bene_df.columns:
    # compute age as of a fixed reference (or use today's date)
    ref_date = pd.Timestamp("2025-01-01")
    bene_df['DOB'] = pd.to_datetime(bene_df['DOB'], errors='coerce')
    bene_df['Age'] = ((ref_date - bene_df['DOB']).dt.days / 365.25).round(1)
else:
    bene_df['Age'] = np.nan

# Detect chronic condition columns (common Kaggle naming like ChronicCond_* or Chronic*)
chronic_candidates = [c for c in bene_df.columns if ('Chronic' in c or 'chronic' in c or 'ChronicCond' in c)]
print("Detected chronic columns (sample):", chronic_candidates[:10])
if len(chronic_candidates) > 0:
    bene_df['ChronicCount'] = bene_df[chronic_candidates].sum(axis=1)
else:
    # If none exist, create a zero column so downstream code doesn't break
    bene_df['ChronicCount'] = 0

# Days to death (if DOD present)
if 'DOD' in bene_df.columns:
    bene_df['DOD'] = pd.to_datetime(bene_df['DOD'], errors='coerce')
    bene_df['DaysToDeath'] = (bene_df['DOD'] - bene_df['DOB']).dt.days
else:
    bene_df['DaysToDeath'] = np.nan

print("Beneficiary features added: Age, ChronicCount, DaysToDeath")
display(bene_df[['BeneID','Age','ChronicCount']].head())

In [ ]:
# ---------------------------
# 6. Claim-level Feature Engineering (Inpatient & Outpatient)
# ---------------------------
print("\n--- Claim-level features (inpatient/outpatient) ---")

# Ensure monetary column numeric
for df, name in [(inp_df, "Inpatient"), (out_df, "Outpatient")]:
    if 'InscClaimAmtReimbursed' in df.columns:
        df['InscClaimAmtReimbursed'] = pd.to_numeric(df['InscClaimAmtReimbursed'], errors='coerce').fillna(0)

# Compute LOS for inpatient (DischargeDt - AdmissionDt)
if ('DischargeDt' in inp_df.columns) and ('AdmissionDt' in inp_df.columns):
    inp_df['LOS'] = (inp_df['DischargeDt'] - inp_df['AdmissionDt']).dt.days
    # clamp negative stays to NaN (we may mark them later)
    inp_df.loc[inp_df['LOS'] < 0, 'LOS'] = np.nan
    print("Inpatient LOS computed.")
else:
    inp_df['LOS'] = np.nan
    print("Inpatient LOS not computed (missing date columns).")

# Mark ClaimType and concat claims
inp_df['ClaimType'] = 'Inpatient'
out_df['ClaimType'] = 'Outpatient'
claims = pd.concat([inp_df, out_df], ignore_index=True, sort=False)
print("Combined claims shape:", claims.shape)

# Merge beneficiary-level derived fields (Age, ChronicCount, DOD) into claims
claims = claims.merge(bene_df[['BeneID','Age','ChronicCount','DOD']], on='BeneID', how='left')

# Flags: claim after death, bad date
claims['ClaimAfterDeath'] = np.where((claims['DOD'].notna()) & (claims['ClaimStartDt'] > claims['DOD']), 1, 0)
claims['BadDate'] = np.where((claims['ClaimStartDt'].notna()) & (claims['ClaimEndDt'].notna()) & (claims['ClaimStartDt'] > claims['ClaimEndDt']), 1, 0)

# Quick check: number of unique providers in claims
print("Unique providers in claims:", claims['Provider'].nunique())